In [1]:
import torch
import numpy as np
import pandas as pd
device = torch.device("cpu")
from transformers import BertTokenizer, BertModel, AutoModelForSequenceClassification, AdamW, get_scheduler
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')                                                     
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=658).to(device)
from sklearn import preprocessing
Label_encoder = preprocessing.LabelEncoder()
Label_encoder.classes_ = np.load('./classes.npy', allow_pickle=True)
model.load_state_dict(torch.load("../../best_model/pytorch_model.bin", map_location=device))

C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining mode

<All keys matched successfully>

In [2]:
def predict_class(text):
    model.to(torch.device('cpu'))
    inputs = tokenizer(text, truncation = True, max_length=100, padding='max_length', return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        return Label_encoder.inverse_transform([predicted_class_id])

In [5]:
predict_class('суперклей')

array(['3919'], dtype=object)

In [4]:
model.to(torch.device('cuda'))
from datasets import load_dataset, load_metric
dataset = load_dataset('csv', data_files={'train': '../../train.csv', 'test': '../../test.csv'})
dataset = dataset.map(lambda e: tokenizer(e['description'], truncation = True, max_length=100, padding='max_length'), batched=True)
pytorch_style_columns = ['input_ids', 'token_type_ids', 'attention_mask', 'labels']  # 
dataset = dataset.remove_columns(set(dataset['train'].features.keys()) - set(pytorch_style_columns))
dataset.set_format(type='torch', columns=pytorch_style_columns, device='cuda') 
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=1)
test_dataloader = DataLoader(dataset['test'], shuffle=False, batch_size=1)
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.metrics import classification_report
Label_encoder = preprocessing.LabelEncoder()
Label_encoder.classes_ = np.load('../../classes.npy', allow_pickle=True)

true = []
preds = []

model.eval()
for batch in tqdm(test_dataloader):
    batch = {k: v.to(torch.device('cuda')) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    true += batch["labels"].detach().cpu().numpy().tolist()
    preds += predictions.detach().cpu().numpy().tolist()

print(classification_report(Label_encoder.inverse_transform(true), Label_encoder.inverse_transform(preds)))

Found cached dataset csv (C:/Users/yeril/.cache/huggingface/datasets/csv/default-c2d1908b4f31630f/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 335.06it/s]
Loading cached processed dataset at C:\Users\yeril\.cache\huggingface\datasets\csv\default-c2d1908b4f31630f\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-cc49031a2d338109.arrow
Loading cached processed dataset at C:\Users\yeril\.cache\huggingface\datasets\csv\default-c2d1908b4f31630f\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-68345e699936c339.arrow
100%|████████████████████████████████████████████████████████████████████████████| 13992/13992 [04:42<00:00, 49.59it/s]
C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:

              precision    recall  f1-score   support

        0304       0.00      0.00      0.00         1
        0305       0.00      0.00      0.00         2
        0604       0.00      0.00      0.00         1
        0701       0.83      1.00      0.91         5
        0711       0.00      0.00      0.00         2
        0801       0.00      0.00      0.00         2
        0802       0.00      0.00      0.00         1
        0803       0.77      1.00      0.87        23
        0804       0.86      1.00      0.92         6
        0805       0.88      0.95      0.91        22
        0806       0.00      0.00      0.00         1
        0808       0.79      1.00      0.88        22
        0813       0.00      0.00      0.00         1
        0830       0.00      0.00      0.00         1
        0902       0.00      0.00      0.00         4
        1012       0.00      0.00      0.00         1
        1018       0.00      0.00      0.00         1
        1202       1.00    

C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: Undefi

In [7]:
a = classification_report(Label_encoder.inverse_transform(true), Label_encoder.inverse_transform(preds))
print(a)

              precision    recall  f1-score   support

        0304       0.00      0.00      0.00         1
        0305       0.00      0.00      0.00         2
        0604       0.00      0.00      0.00         1
        0701       0.83      1.00      0.91         5
        0711       0.00      0.00      0.00         2
        0801       0.00      0.00      0.00         2
        0802       0.00      0.00      0.00         1
        0803       0.77      1.00      0.87        23
        0804       0.86      1.00      0.92         6
        0805       0.88      0.95      0.91        22
        0806       0.00      0.00      0.00         1
        0808       0.79      1.00      0.88        22
        0813       0.00      0.00      0.00         1
        0830       0.00      0.00      0.00         1
        0902       0.00      0.00      0.00         4
        1012       0.00      0.00      0.00         1
        1018       0.00      0.00      0.00         1
        1202       1.00    

C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: Undefi